In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import subprocess


def get_git_root():
    """Get the root directory of the git repository"""
    try:
        git_root = (
            subprocess.check_output(
                ["git", "rev-parse", "--show-toplevel"], stderr=subprocess.DEVNULL
            )
            .strip()
            .decode("utf-8")
        )
        return git_root
    except (subprocess.CalledProcessError, FileNotFoundError):
        return None


# Change to git root
git_root = get_git_root()
if git_root:
    os.chdir(git_root)
    print(f"Changed to git root: {os.getcwd()}")
else:
    print("Not in a git repository or git not found")

Changed to git root: /Users/sherman/GitHub/CSE-6748


In [3]:
import pandas as pd

import src.features.illuminate as illuminate
import src.features.index as index
import src.features.milestone as milestone
import src.features.rit as rit
import src.features.student as student
import src.features.targets as targets
from src.feasy.sparkle import Sparkle
from src.utils import create_student_features, get_feature_functions_from_module

pd.set_option("display.max_columns", None)
pd.options.mode.copy_on_write = True

In [4]:
student_profile = pd.read_pickle("data/cleaned/student_profile_dataset.pkl")
student_profile.head()

,mask_studentpersonkey,GradeLevel,CurrentEnrollment,FinalPrimaryEnrollmentForYearFlag,RetainedFlag,EnrollmentReasonDesc,WithDrawalReasonDesc,FirstDayOfSchoolYear,SchoolYearNumberFall,SchoolYearNumberSpring,SchoolStatusDesc,SchoolDetailReportName,SchoolDetailLevelDesc,SchoolStatusDescNextYear,SchoolDetailReportNameNextYear,SchoolDetailLevelDescNextYear,ActiveGiftedStudentResultRecordFlag,grad,milestone,illuminate,sat,rit
2,107055,6,N,Y,N,Transferred from another GA district,Year End,2023-07-19,2023,2024,Open,Woodland Middle,Middle School,Open,Woodland Middle,Middle School,NaN,NaN,NestedData(SubjectDesc=['English Language Arts...,NestedData(responsedatevalue=[datetime.date(20...,NaN,NaN
3,114271,12,N,Y,Y,Continuing in same school,High School Graduation,2021-07-24,2021,2022,Open,Johns Creek High,High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,117427,12,N,Y,N,Continuing in same school,High School Graduation,2021-07-24,2021,2022,Open,Northview High,High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,117773,12,N,Y,N,Continuing in same school,Court Order or Legal Requirement,2021-07-24,2021,2022,Open,Centennial High,High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,117995,12,N,Y,N,Continuing in same school,High School Graduation,2021-07-24,2021,2022,Open,Roswell High,High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# fmt:off
def analyze_dataset(df):
    """Comprehensive analysis of the merged student dataset"""

    import itertools

    import pandas as pd
    from IPython.display import display

    with pd.option_context(
        'display.max_columns', None,
        'display.width', None,
        'display.max_colwidth', None,
        'display.expand_frame_repr', False
    ):

        print("COMPREHENSIVE STUDENT DATASET ANALYSIS")
        print("=" * 70)

        # Basic metrics
        total_records = len(df)
        unique_students = df["mask_studentpersonkey"].nunique()

        # Current students filter - get IDs of students who have at least one current enrollment record
        current_student_ids = df[df["CurrentEnrollment"] == "Y"]["mask_studentpersonkey"].unique()
        # Get ALL records for current students (including historical records)
        current_students_df = df[df["mask_studentpersonkey"].isin(current_student_ids)]
        total_current_records = len(current_students_df)
        unique_current_students = len(current_student_ids)

        # Dataset Overview
        overview_data = {
            'Metric': [
                'Total Records',
                'Unique Students',
                'Average Records per Student',
                'Current Student Records',
                'Unique Current Students',
                'Average Records per Current Student'
            ],
            'Value': [
                f"{total_records:,}",
                f"{unique_students:,}",
                f"{total_records/unique_students:.2f}",
                f"{total_current_records:,}",
                f"{unique_current_students:,}",
                f"{total_current_records/unique_current_students:.2f}" if unique_current_students > 0 else "N/A"
            ]
        }

        print("\nDATASET OVERVIEW:")
        display(pd.DataFrame(overview_data))

        # Enrollment Coverage
        records_with_enrollment = df[~df["GradeLevel"].isnull()]
        records_missing_enrollment = df[df["GradeLevel"].isnull()]
        current_records_with_enrollment = current_students_df[~current_students_df["GradeLevel"].isnull()]
        current_records_missing_enrollment = current_students_df[current_students_df["GradeLevel"].isnull()]

        enrollment_data = {
            'Metric': [
                'Records with Enrollment',
                'Records missing Enrollment',
                'Students with Enrollment',
                'Students missing Enrollment'
            ],
            'All Students': [
                f"{len(records_with_enrollment):,} ({len(records_with_enrollment)/total_records*100:.1f}%)",
                f"{len(records_missing_enrollment):,} ({len(records_missing_enrollment)/total_records*100:.1f}%)",
                f"{records_with_enrollment['mask_studentpersonkey'].nunique():,} ({records_with_enrollment['mask_studentpersonkey'].nunique()/unique_students*100:.1f}%)",
                f"{records_missing_enrollment['mask_studentpersonkey'].nunique():,} ({records_missing_enrollment['mask_studentpersonkey'].nunique()/unique_students*100:.1f}%)"
            ],
            'Current Students': [
                f"{len(current_records_with_enrollment):,} ({len(current_records_with_enrollment)/total_current_records*100:.1f}%)" if total_current_records > 0 else "0 (0.0%)",
                f"{len(current_records_missing_enrollment):,} ({len(current_records_missing_enrollment)/total_current_records*100:.1f}%)" if total_current_records > 0 else "0 (0.0%)",
                f"{current_records_with_enrollment['mask_studentpersonkey'].nunique():,} ({current_records_with_enrollment['mask_studentpersonkey'].nunique()/unique_current_students*100:.1f}%)" if unique_current_students > 0 else "0 (0.0%)",
                f"{current_records_missing_enrollment['mask_studentpersonkey'].nunique():,} ({current_records_missing_enrollment['mask_studentpersonkey'].nunique()/unique_current_students*100:.1f}%)" if unique_current_students > 0 else "0 (0.0%)"
            ]
        }

        print("\nENROLLMENT DATA COVERAGE:")
        display(pd.DataFrame(enrollment_data))

        # Assessment Coverage
        assessments = {
            "Graduation Data": "grad",
            "GA Milestones": "milestone",
            "Illuminate": "illuminate",
            "SAT Scores": "sat",
            "RIT Scores": "rit",
        }

        assessment_data = []
        for name, col in assessments.items():
            # All students
            records_with_data = df[~df[col].isnull()]
            record_count = len(records_with_data)
            student_count = records_with_data["mask_studentpersonkey"].nunique()

            # Current students - use ALL records for current students (including historical)
            current_records_with_data = current_students_df[~current_students_df[col].isnull()]
            current_record_count = len(current_records_with_data)
            # Count unique current students who have this assessment data
            current_student_count = current_records_with_data["mask_studentpersonkey"].nunique()

            assessment_data.append({
                'Assessment': name,
                'All Students - Records': f"{record_count:,} ({record_count/total_records*100:.1f}%)",
                'All Students - Students': f"{student_count:,} ({student_count/unique_students*100:.1f}%)",
                'Current Students - Records': f"{current_record_count:,} ({current_record_count/total_current_records*100:.1f}%)" if total_current_records > 0 else "0 (0.0%)",
                'Current Students - Students': f"{current_student_count:,} ({current_student_count/unique_current_students*100:.1f}%)" if unique_current_students > 0 else "0 (0.0%)"
            })

        print("\nASSESSMENT DATA COVERAGE:")
        display(pd.DataFrame(assessment_data))

        # SAT Combination Analysis (All Students Only)
        non_sat_assessments = {k: v for k, v in assessments.items() if v != "sat"}
        sat_combinations = []

        # SAT + each individual assessment
        for name, col in non_sat_assessments.items():
            sat_combinations.append((f"SAT + {name}", ["sat", col]))

        # SAT + pairs
        for combo in itertools.combinations(non_sat_assessments.items(), 2):
            names = [item[0] for item in combo]
            cols = [item[1] for item in combo]
            sat_combinations.append((f"SAT + {' + '.join(names)}", ["sat"] + cols))

        # SAT + triplets
        for combo in itertools.combinations(non_sat_assessments.items(), 3):
            names = [item[0] for item in combo]
            cols = [item[1] for item in combo]
            sat_combinations.append((f"SAT + {' + '.join(names)}", ["sat"] + cols))

        # SAT + all
        all_names = list(non_sat_assessments.keys())
        all_cols = list(non_sat_assessments.values())
        sat_combinations.append((f"SAT + {' + '.join(all_names)}", ["sat"] + all_cols))

        sat_combo_data = []
        for combo_name, combo_cols in sat_combinations:
            # All students only
            mask = df[combo_cols[0]].notna()
            for col in combo_cols[1:]:
                mask = mask & df[col].notna()
            students_with_combo = df[mask]["mask_studentpersonkey"].nunique()

            sat_combo_data.append({
                'Combination': combo_name,
                'All Students': f"{students_with_combo:,} ({students_with_combo/unique_students*100:.1f}%)"
            })

        print("\nSAT COMBINATION ANALYSIS:")
        display(pd.DataFrame(sat_combo_data))

        # NON-SAT Combination Analysis (Current Students Only)
        if unique_current_students > 0:
            non_sat_combinations = []

            # Individual assessments (excluding SAT)
            for name, col in non_sat_assessments.items():
                non_sat_combinations.append((name, [col]))

            # Pairs of non-SAT assessments
            for combo in itertools.combinations(non_sat_assessments.items(), 2):
                names = [item[0] for item in combo]
                cols = [item[1] for item in combo]
                combo_name = f"{' + '.join(names)}"
                non_sat_combinations.append((combo_name, cols))

            # Triplets of non-SAT assessments
            for combo in itertools.combinations(non_sat_assessments.items(), 3):
                names = [item[0] for item in combo]
                cols = [item[1] for item in combo]
                combo_name = f"{' + '.join(names)}"
                non_sat_combinations.append((combo_name, cols))

            # All non-SAT assessments
            all_non_sat_names = list(non_sat_assessments.keys())
            all_non_sat_cols = list(non_sat_assessments.values())
            non_sat_combinations.append((f"{' + '.join(all_non_sat_names)}", all_non_sat_cols))

            current_combo_data = []
            for combo_name, combo_cols in non_sat_combinations:
                # Current students only - using ALL records for current students (including historical)
                current_mask = current_students_df[combo_cols[0]].notna()
                for col in combo_cols[1:]:
                    current_mask = current_mask & current_students_df[col].notna()

                current_students_with_combo = current_students_df[current_mask]["mask_studentpersonkey"].nunique()

                current_combo_data.append({
                    'Combination': combo_name,
                    'Current Students': f"{current_students_with_combo:,} ({current_students_with_combo/unique_current_students*100:.1f}%)"
                })

            print("\nNON-SAT COMBINATION ANALYSIS (CURRENT STUDENTS):")
            display(pd.DataFrame(current_combo_data))
        else:
            print("\nNON-SAT COMBINATION ANALYSIS (CURRENT STUDENTS):")
            print("No current students found in dataset.")

# fmt: on

analyze_dataset(student_profile)

COMPREHENSIVE STUDENT DATASET ANALYSIS

DATASET OVERVIEW:


,Metric,Value
0,Total Records,"385,962"
1,Unique Students,"150,306"
2,Average Records per Student,2.57
3,Current Student Records,"272,009"
4,Unique Current Students,"87,376"
5,Average Records per Current Student,3.11



ENROLLMENT DATA COVERAGE:


,Metric,All Students,Current Students
0,Records with Enrollment,"385,962 (100.0%)","272,009 (100.0%)"
1,Records missing Enrollment,0 (0.0%),0 (0.0%)
2,Students with Enrollment,"150,306 (100.0%)","87,376 (100.0%)"
3,Students missing Enrollment,0 (0.0%),0 (0.0%)



ASSESSMENT DATA COVERAGE:


,Assessment,All Students - Records,All Students - Students,Current Students - Records,Current Students - Students
0,Graduation Data,"24,733 (6.4%)","24,733 (16.5%)","24,729 (9.1%)","24,729 (28.3%)"
1,GA Milestones,"168,969 (43.8%)","86,697 (57.7%)","122,564 (45.1%)","53,726 (61.5%)"
2,Illuminate,"267,037 (69.2%)","120,259 (80.0%)","209,360 (77.0%)","80,425 (92.0%)"
3,SAT Scores,"24,125 (6.3%)","19,424 (12.9%)","8,823 (3.2%)","6,950 (8.0%)"
4,RIT Scores,"114,814 (29.7%)","81,252 (54.1%)","82,783 (30.4%)","56,324 (64.5%)"



SAT COMBINATION ANALYSIS:


,Combination,All Students
0,SAT + Graduation Data,73 (0.0%)
1,SAT + GA Milestones,"6,892 (4.6%)"
2,SAT + Illuminate,"5,395 (3.6%)"
3,SAT + RIT Scores,"11,701 (7.8%)"
4,SAT + Graduation Data + GA Milestones,62 (0.0%)
5,SAT + Graduation Data + Illuminate,16 (0.0%)
6,SAT + Graduation Data + RIT Scores,51 (0.0%)
7,SAT + GA Milestones + Illuminate,"3,904 (2.6%)"
8,SAT + GA Milestones + RIT Scores,"4,462 (3.0%)"
9,SAT + Illuminate + RIT Scores,"3,240 (2.2%)"



NON-SAT COMBINATION ANALYSIS (CURRENT STUDENTS):


,Combination,Current Students
0,Graduation Data,"24,729 (28.3%)"
1,GA Milestones,"53,726 (61.5%)"
2,Illuminate,"80,425 (92.0%)"
3,RIT Scores,"56,324 (64.5%)"
4,Graduation Data + GA Milestones,"16,901 (19.3%)"
5,Graduation Data + Illuminate,"19,327 (22.1%)"
6,Graduation Data + RIT Scores,"15,108 (17.3%)"
7,GA Milestones + Illuminate,"51,346 (58.8%)"
8,GA Milestones + RIT Scores,"37,504 (42.9%)"
9,Illuminate + RIT Scores,"52,001 (59.5%)"


In [6]:
def create_sat_filtered_training_data(student_profile):
    """
    Filter students to include records only up to and including the year they first took SAT.

    For each student:
    - Find the earliest year they took SAT
    - Include all records for that student from the earliest year up to the SAT year
    - Exclude any records after the SAT year

    Examples:
    Student A: Grade 10 (SAT), Grade 11 (No SAT), Grade 12 (No SAT) -> Keep only Grade 10
    Student B: Grade 10 (No SAT), Grade 11 (SAT), Grade 12 (No SAT) -> Keep Grade 10-11
    """

    print("Creating SAT-filtered training dataset...")

    # First, identify students who have SAT scores
    students_with_sat = student_profile[student_profile["sat"].notna()].copy()

    if students_with_sat.empty:
        print("No students with SAT scores found.")
        return pd.DataFrame()

    print(
        f"Found {students_with_sat['mask_studentpersonkey'].nunique()} students with SAT scores"
    )

    # For each student, find their earliest SAT year
    sat_first_year = (
        students_with_sat.groupby("mask_studentpersonkey")["SchoolYearNumberFall"]
        .min()
        .reset_index()
        .rename(columns={"SchoolYearNumberFall": "first_sat_year"})
    )

    print("SAT year distribution:")
    display(sat_first_year[["first_sat_year"]].value_counts().sort_index())

    # Merge back with the full student profile to get all historical records
    student_profile_with_sat_year = pd.merge(
        student_profile,
        sat_first_year,
        on="mask_studentpersonkey",
        how="inner",
    )

    print(f"Total records for SAT students: {len(student_profile_with_sat_year)}")

    # Filter to include only records up to and including the first SAT year
    filtered_data = student_profile_with_sat_year[
        student_profile_with_sat_year["SchoolYearNumberFall"]
        <= student_profile_with_sat_year["first_sat_year"]
    ].copy()

    print(f"Records after filtering to SAT cutoff: {len(filtered_data)}")

    # Remove students who have no assessment data on the latest year
    # First, find each student's latest year
    latest_year_per_student = (
        filtered_data.groupby("mask_studentpersonkey")["SchoolYearNumberFall"]
        .max()
        .reset_index()
    )

    latest_year_per_student.columns = ["mask_studentpersonkey", "latest_year"]

    # Get records for each student's latest year only
    latest_year_records = pd.merge(
        filtered_data,
        latest_year_per_student,
        left_on=["mask_studentpersonkey", "SchoolYearNumberFall"],
        right_on=["mask_studentpersonkey", "latest_year"],
    )

    # Check which students have assessments in their latest year
    latest_year_has_illuminate = latest_year_records["illuminate"].notna()
    latest_year_has_milestone = latest_year_records["milestone"].notna()
    latest_year_has_rit = latest_year_records["rit"].notna()

    latest_year_has_assessments = (
        latest_year_has_illuminate | latest_year_has_milestone | latest_year_has_rit
    )

    # Get students who have assessments in their latest year
    students_with_latest_assessments = latest_year_records[latest_year_has_assessments][
        "mask_studentpersonkey"
    ].unique()

    # Filter filtered_data to keep only these students
    final_data = filtered_data[
        filtered_data["mask_studentpersonkey"].isin(students_with_latest_assessments)
    ].copy()

    print(f"Final records after assessment filter: {len(final_data)}")
    print(f"Final unique students: {final_data['mask_studentpersonkey'].nunique()}")

    # Show grade level distribution for each student's latest year
    print("\nGrade level distribution (latest year per student):")
    latest_year_per_student = (
        final_data.groupby("mask_studentpersonkey")["SchoolYearNumberFall"]
        .max()
        .reset_index()
    )

    latest_records = pd.merge(
        final_data,
        latest_year_per_student,
        on=["mask_studentpersonkey", "SchoolYearNumberFall"],
    )

    grade_dist = latest_records.groupby(["GradeLevel"])[
        ["mask_studentpersonkey"]
    ].nunique()

    display(grade_dist)

    return final_data.drop(columns=["first_sat_year"])


# Apply the new filtering logic
data = create_sat_filtered_training_data(student_profile)

Creating SAT-filtered training dataset...
Found 19424 students with SAT scores
SAT year distribution:


first_sat_year
2021              5894
2022              4434
2023              4766
2024              4330
Name: count, dtype: int64

Total records for SAT students: 52134
Records after filtering to SAT cutoff: 44128
Final records after assessment filter: 32497
Final unique students: 13557

Grade level distribution (latest year per student):


,mask_studentpersonkey
GradeLevel,
10,822
11,6977
12,5675
7,2
8,3
9,78


In [7]:
index_feat_funcs = get_feature_functions_from_module(index)
student_feat_funcs = get_feature_functions_from_module(student)
illuminate_feat_funcs = get_feature_functions_from_module(illuminate)
milestone_feat_funcs = get_feature_functions_from_module(milestone)
rit_feat_funcs = get_feature_functions_from_module(rit)
targets_feat_funcs = get_feature_functions_from_module(targets)

features = (
    index_feat_funcs
    + student_feat_funcs
    + illuminate_feat_funcs
    + milestone_feat_funcs
    + rit_feat_funcs
    + targets_feat_funcs
)

In [8]:
sparkle = Sparkle(features).source(data, from_dataframe=True)
features = sparkle.to_pandas()
features.head()

,SchoolYearFall,mask_studentpersonkey,RetainedFlag,GradeLevel,is_gifted,grade_mode_numeric,improvement_first_to_last,is_on_sat_grade_level,last_percent_correct,low_mastery_rate_below_50,mastery_rate_above_80,math_improvement_first_to_last,math_low_mastery_rate_below_50,math_mastery_rate_above_80,math_mean_item_pct_correct,math_percent_extension,math_percent_reteach,math_percent_review_practice,math_slope_percent_correct_over_time,math_std_item_pct_correct,math_total_points_earned,math_total_points_possible,math_weighted_percent_correct,mean_item_percent_correct,mean_standard_percent_correct,mean_std_pct_ela,mean_std_pct_math,num_assessments,num_items,percent_extension,percent_reteach,percent_review_practice,slope_percent_correct_over_time,std_item_percent_correct,total_points_earned,total_points_possible,verbal_improvement_first_to_last,verbal_low_mastery_rate_below_50,verbal_mastery_rate_above_80,verbal_mean_item_pct_correct,verbal_percent_extension,verbal_percent_reteach,verbal_percent_review_practice,verbal_slope_percent_correct_over_time,verbal_std_item_pct_correct,verbal_total_points_earned,verbal_total_points_possible,verbal_weighted_percent_correct,weighted_overall_percent_correct,achievement_level_mean_ela,achievement_level_mean_math,lexile_mean_ela,lexile_mean_math,mean_achievement_level_all,mean_lexile_score_all,mean_scale_score_all,num_subjects_tested,num_test_days,num_unique_test_dates,pct_proficient_all,pct_proficient_ela,pct_proficient_math,scale_score_improvement_all,scale_score_improvement_ela,scale_score_improvement_math,scale_score_mean_ela,scale_score_mean_math,scale_score_slope_all,scale_score_slope_ela,scale_score_slope_math,scale_score_std_ela,scale_score_std_math,std_scale_score_all,count_quintile_avg,count_quintile_hiavg,count_quintile_high,count_quintile_low,count_quintile_lowavg,max_rit_score,mean_percent_correct,mean_percentile,mean_rit_score,mean_test_duration,min_rit_score,num_tests,pct_quintile_high,std_percent_correct,std_percentile,std_rit_score,std_test_duration,sat_math_score,sat_verbal_score
0,2022,1389182,N,9,N,0,0.00,0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0,0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,4.0,0.0,623.0,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,2,0,0,253.0,50.666667,78.666667,242.000000,58.666667,232.0,3,0.666667,1.699673,15.369523,8.602325,18.116904,NaN,NaN
1,2023,1389182,N,10,N,0,0.00,0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0,0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2,0,0,247.0,51.000000,82.500000,243.000000,65.500000,239.0,2,1.000000,2.000000,1.500000,4.000000,9.500000,500.0,480.0
2,2022,1390795,N,9,Y,9,11.76,0,100.0,0.235955,0.730337,11.76,0.313433,0.656716,71.757463,0.656716,0.343284,0.0,0.140152,22.851185,912.0,1295.0,0.704247,78.176966,79.138652,98.4850,72.786119,89,89,0.730337,0.269663,0.00,0.229855,22.821853,1256.0,1647.0,0.00,0.00,0.954545,97.727273,0.954545,0.045455,0.00,0.000000,3.006536,344.0,352.0,0.977273,0.762599,0.0,3.0,0.0,0.0,3.0,0.0,577.5,2,1,1,1.0,0.0,1.0,3.0,0.0,0.0,0.0,576.0,3.0,0.0,0.0,0.0,0.0,1.5,0,2,1,0,0,253.0,53.333333,82.666667,243.333333,82.000000,234.0,3,0.333333,4.642796,4.496913,7.760298,8.640988,NaN,NaN
3,2023,1390795,N,10,Y,0,0.00,0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0,0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0

In [9]:
STUDENT = "mask_studentpersonkey"
YEAR = "SchoolYearFall"
TARGET_MATH = "sat_math_score"
TARGET_VERBAL = "sat_verbal_score"

In [10]:
X_index, X = create_student_features(
    features,
    student_col=STUDENT,
    year_col=YEAR,
    target_cols=[TARGET_MATH, TARGET_VERBAL],
)

Y = (
    features.sort_values([STUDENT, YEAR], ascending=True)
    .groupby(STUDENT)[[TARGET_MATH, TARGET_VERBAL]]
    .last()
    .reset_index(drop=True)
)

In [11]:
display(X_index.head())
display(X_index.shape)
display(X.head())
display(X.shape)
display(Y.head())
display(Y.shape)

,mask_studentpersonkey
0,1389182
1,1390795
2,1392398
3,1395568
4,1395578


(13557, 1)

,RetainedFlag_latest,GradeLevel_latest,is_gifted_latest,grade_mode_numeric_latest,improvement_first_to_last_latest,is_on_sat_grade_level_latest,last_percent_correct_latest,low_mastery_rate_below_50_latest,mastery_rate_above_80_latest,math_improvement_first_to_last_latest,math_low_mastery_rate_below_50_latest,math_mastery_rate_above_80_latest,math_mean_item_pct_correct_latest,math_percent_extension_latest,math_percent_reteach_latest,math_percent_review_practice_latest,math_slope_percent_correct_over_time_latest,math_std_item_pct_correct_latest,math_total_points_earned_latest,math_total_points_possible_latest,math_weighted_percent_correct_latest,mean_item_percent_correct_latest,mean_standard_percent_correct_latest,mean_std_pct_ela_latest,mean_std_pct_math_latest,num_assessments_latest,num_items_latest,percent_extension_latest,percent_reteach_latest,percent_review_practice_latest,slope_percent_correct_over_time_latest,std_item_percent_correct_latest,total_points_earned_latest,total_points_possible_latest,verbal_improvement_first_to_last_latest,verbal_low_mastery_rate_below_50_latest,verbal_mastery_rate_above_80_latest,verbal_mean_item_pct_correct_latest,verbal_percent_extension_latest,verbal_percent_reteach_latest,verbal_percent_review_practice_latest,verbal_slope_percent_correct_over_time_latest,verbal_std_item_pct_correct_latest,verbal_total_points_earned_latest,verbal_total_points_possible_latest,verbal_weighted_percent_correct_latest,weighted_overall_percent_correct_latest,achievement_level_mean_ela_latest,achievement_level_mean_math_latest,lexile_mean_ela_latest,lexile_mean_math_latest,mean_achievement_level_all_latest,mean_lexile_score_all_latest,mean_scale_score_all_latest,num_subjects_tested_latest,num_test_days_latest,num_unique_test_dates_latest,pct_proficient_all_latest,pct_proficient_ela_latest,pct_proficient_math_latest,scale_score_improvement_all_latest,scale_score_improvement_ela_latest,scale_score_improvement_math_latest,scale_score_mean_ela_latest,scale_score_mean_math_latest,scale_score_slope_all_latest,scale_score_slope_ela_latest,scale_score_slope_math_latest,scale_score_std_ela_latest,scale_score_std_math_latest,std_scale_score_all_latest,count_quintile_avg_latest,count_quintile_hiavg_latest,count_quintile_high_latest,count_quintile_low_latest,count_quintile_lowavg_latest,max_rit_score_latest,mean_percent_correct_latest,mean_percentile_latest,mean_rit_score_latest,mean_test_duration_latest,min_rit_score_latest,num_tests_latest,pct_quintile_high_latest,std_percent_correct_latest,std_percentile_latest,std_rit_score_latest,std_test_duration_latest,grade_mode_numeric_delta,improvement_first_to_last_delta,is_on_sat_grade_level_delta,last_percent_correct_delta,low_mastery_rate_below_50_delta,mastery_rate_above_80_delta,math_improvement_first_to_last_delta,math_low_mastery_rate_below_50_delta,math_mastery_rate_above_80_delta,math_mean_item_pct_correct_delta,math_percent_extension_delta,math_percent_reteach_delta,math_percent_review_practice_delta,math_slope_percent_correct_over_time_delta,math_std_item_pct_correct_delta,math_total_points_earned_delta,math_total_points_possible_delta,math_weighted_percent_correct_delta,mean_item_percent_correct_delta,mean_standard_percent_correct_delta,mean_std_pct_ela_delta,mean_std_pct_math_delta,num_assessments_delta,num_items_delta,percent_extension_delta,percent_reteach_delta,percent_review_practice_delta,slope_percent_correct_over_time_delta,std_item_percent_correct_delta,total_points_earned_delta,total_points_possible_delta,verbal_improvement_first_to_last_delta,verbal_low_mastery_rate_below_50_delta,verbal_mastery_rate_above_80_delta,verbal_mean_item_pct_correct_delta,verbal_percent_extension_delta,verbal_percent_reteach_delta,verbal_percent_review_practice_delta,verbal_slope_percent_correct_over_time_delta,verbal_std_item_pct_correct_delta,verbal_total_points_earned_delta,verbal_total_points_possible_delta,verbal_weighted_percent_correct_delta,weighted_overall_percent_correc

(13557, 259)

,sat_math_score,sat_verbal_score
0,500.0,480.0
1,560.0,550.0
2,610.0,680.0
3,560.0,560.0
4,510.0,450.0


(13557, 2)

In [12]:
X_index.to_parquet("data/cleaned/X_index.parquet")
X.to_parquet("data/cleaned/X.parquet")
Y.to_parquet("data/cleaned/Y.parquet")